# How to train your model on AI Platform with HP tuning.
Using HP Tuning for training can be done in a few steps:
1. Create your python model file
    1. Add argument parsing for the hyperparameter values. (These values are chosen for you in this notebook)
    1. Add code to download your data from [Google Cloud Storage](https://cloud.google.com/storage) so that AI Platform can use it
    1. Add code to track the performance of your hyperparameter values.
    1. Add code to export and save the model to [Google Cloud Storage](https://cloud.google.com/storage) once AI Platform finishes training the model
1. Prepare a package
1. Submit the training job

# Prerequisites
Before you jump in, let’s cover some of the different tools you’ll be using to get HP tuning up and running on AI Platform. 

[Google Cloud Platform](https://cloud.google.com/) lets you build and host applications and websites, store data, and analyze data on Google's scalable infrastructure.

[AI Platform](https://cloud.google.com/ml-engine/) is a managed service that enables you to easily build machine learning models that work on any type of data, of any size.

[Google Cloud Storage](https://cloud.google.com/storage/) (GCS) is a unified object storage for developers and enterprises, from live data serving to data analytics/ML to data archiving.

[Cloud SDK](https://cloud.google.com/sdk/) is a command line tool which allows you to interact with Google Cloud products. In order to run this notebook, make sure that Cloud SDK is [installed](https://cloud.google.com/sdk/downloads) in the same environment as your Jupyter kernel.

[Overview of Hyperparameter Tuning](https://cloud.google.com/ml-engine/docs/tensorflow/hyperparameter-tuning-overview) - Hyperparameter tuning takes advantage of the processing infrastructure of Google Cloud Platform to test different hyperparameter configurations when training your model.

# Part 0: Setup
* [Create a project on GCP](https://cloud.google.com/resource-manager/docs/creating-managing-projects)
* [Create a Google Cloud Storage Bucket](https://cloud.google.com/storage/docs/quickstart-console)
* [Enable AI Platform Training and Prediction and Compute Engine APIs](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component&_ga=2.217405014.1312742076.1516128282-1417583630.1516128282)
* [Install Cloud SDK](https://cloud.google.com/sdk/downloads)
* [Install XGBoost](https://xgboost.readthedocs.io/en/latest/build.html) [Optional: used if running locally]
* [Install pandas](https://pandas.pydata.org/pandas-docs/stable/install.html) [Optional: used if running locally]
* [Install cloudml-hypertune](https://pypi.org/project/cloudml-hypertune/) [Optional: used if running locally]

These variables will be needed for the following steps.
* `TRAINER_PACKAGE_PATH <./hp_tuning>` - A packaged training application that will be staged in a Google Cloud Storage location. The model file created below is placed inside this package path.
* `MAIN_TRAINER_MODULE <hp_tuning.train>` - Tells AI Platform which file to execute. This is formatted as follows <folder_name.python_file_name>
* `JOB_DIR <gs://$BUCKET_ID/xgboost_learn_job_dir>` - The path to a Google Cloud Storage location to use for job output.
* `RUNTIME_VERSION <1.9>` - The version of AI Platform to use for the job. If you don't specify a runtime version, the training service uses the default AI Platform runtime version 1.0. [See the list of runtime versions for more information](https://cloud.google.com/ml-engine/docs/tensorflow/runtime-version-list).
* `PYTHON_VERSION <3.5>` - The Python version to use for the job. Python 3.5 is available with runtime version 1.4 or greater. If you don't specify a Python version, the training service uses Python 2.7.
* `HPTUNING_CONFIG <hptuning_config.yaml>` - Path to the job configuration file.

** Replace: **
* `PROJECT_ID <YOUR_PROJECT_ID>` - with your project's id. Use the PROJECT_ID that matches your Google Cloud Platform project.
* `BUCKET_ID <YOUR_BUCKET_ID>` - with the bucket id you created above.
* `JOB_DIR <gs://YOUR_BUCKET_ID/xgboost_job_dir>` - with the bucket id you created above.
* `REGION <REGION>` - select a region from [here](https://cloud.google.com/ml-engine/docs/regions) or use the default '`us-central1`'. The region is where the model will be deployed.

In [1]:
# Replace <PROJECT_ID> and <BUCKET_ID> with proper Project and Bucket ID's:
%env PROJECT_ID mwe-sanofi-ml-workshop
%env BUCKET_ID ml-lending-club-demo
%env REGION us-central1
%env TRAINER_PACKAGE_PATH ./lending_club_hp_tuning
%env MAIN_TRAINER_MODULE lending_club_hp_tuning.train
%env RUNTIME_VERSION 1.15
%env PYTHON_VERSION 3.7
%env HPTUNING_CONFIG hptuning_config.yaml

env: PROJECT_ID=mwe-sanofi-ml-workshop
env: BUCKET_ID=ml-lending-club-demo
env: REGION=us-central1
env: TRAINER_PACKAGE_PATH=./lending_club_hp_tuning
env: MAIN_TRAINER_MODULE=lending_club_hp_tuning.train
env: RUNTIME_VERSION=1.15
env: PYTHON_VERSION=3.7
env: HPTUNING_CONFIG=hptuning_config.yaml


In [2]:
# Create bucket if it doesn't exist.
!gsutil mb gs://${BUCKET_ID}

# Training file must be in a cloud storage bucket before training runs.
!gsutil cp 'Lending Club Data - DR_Demo_Lending_Club.tsv' gs://${BUCKET_ID}

Creating gs://ml-lending-club-demo/...
ServiceException: 409 Bucket ml-lending-club-demo already exists.
Copying file://Lending Club Data - DR_Demo_Lending_Club.tsv [Content-Type=text/tab-separated-values]...
/ [1 files][  4.2 MiB/  4.2 MiB]                                                
Operation completed over 1 objects/4.2 MiB.                                      


In [3]:
from pathlib import Path
Path("./lending_club_hp_tuning/").mkdir(exist_ok=True)

# Part 1: Create your python model file

First, we'll create the python model file (provided below) that we'll upload to AI Platform. This is similar to your normal process for creating a XGBoost model. However, there are a few key differences:
1. Downloading the data from GCS at the start of your file, so that AI Platform can access the data.
1. Exporting/saving the model to GCS at the end of your file, so that you can use it for predictions.
1. Define a command-line argument in your main training module for each tuned hyperparameter.
1. Use the value passed in those arguments to set the corresponding hyperparameter in your application's XGBoost code.
1. Use `cloudml-hypertune` to track your training jobs metrics.

The code in this file first handles the hyperparameters passed to the file from AI Platform. Then it loads the data into a pandas DataFrame that can be used by XGBoost. Then the model is fit against the training data and the metrics for that data are shared with AI Platform. Lastly, Python's built in pickle library is used to save the model to a file that can be uploaded to [AI Platform's prediction service](https://cloud.google.com/ml-engine/docs/scikit/getting-predictions#deploy_models_and_versions).

Note: In normal practice you would want to test your model locally on a small dataset to ensure that it works, before using it with your larger dataset on AI Platform. This avoids wasted time and costs.

### Setup the imports and helper functions

In [4]:
%%writefile ./lending_club_hp_tuning/train.py

import argparse
import datetime
import os
import pandas as pd
import subprocess
import pickle

from google.cloud import storage
import hypertune
import xgboost as xgb

Overwriting ./lending_club_hp_tuning/train.py


### Load the hyperparameter values that are passed to the model during training.

In this tutorial, the Lasso regressor is used, because it has several parameters that can be used to help demonstrate how to choose HP tuning values. (The range of values are set below in the configuration file for the HP tuning values.)

In [5]:
%%writefile -a ./lending_club_hp_tuning/train.py

parser = argparse.ArgumentParser()
parser.add_argument(
    '--job-dir',  # handled automatically by AI Platform
    help='GCS location to write checkpoints and export models',
    required=True
)
parser.add_argument(
    '--max_depth',  # Specified in the config file
    help='Maximum depth of the XGBoost tree. default: 3',
    default=3,
    type=int
)
parser.add_argument(
    '--num_boost_round',  # Specified in the config file
    help='Number of boosting iterations. default: 100',
    default=100,
    type=int
)
parser.add_argument(
    '--booster',  # Specified in the config file
    help='which booster to use: gbtree, gblinear or dart. default: gbtree',
    default='gbtree',
    type=str
)
parser.add_argument(
    '--project-id',
    type=str,
    default='mwe-sanofi-ml-workshop',
    help='The GCP Project ID'
)
parser.add_argument(
    '--bucket-name',
    type=str,
    default='ml-lending-club-demo',
    help='The Cloud Storage bucket to be used for process artifacts'
)

args = parser.parse_args()



Appending to ./lending_club_hp_tuning/train.py


### Add code to download the data from GCS
In this case, using the publicly hosted data,AI Platform will then be able to use the data when training your model.

In [6]:
%%writefile -a ./lending_club_hp_tuning/train.py

# Bucket holding the lending club data
bucket = storage.Client().bucket(args.bucket_name)
# Path to the data inside the bucket
blob = bucket.blob('Lending Club Data - DR_Demo_Lending_Club.tsv')
# Download the data
blob.download_to_filename('Lending Club Data - DR_Demo_Lending_Club.tsv')


import pandas as pd
import numpy as np

# Return first three digits of zip code.
z3 = lambda z : int(z[:3])

def dateparse(dt):
    # test for empty string '' or NaT
    if not dt or pd.isnull(dt):
        return pd.NaT
    m, d, y = map(int, dt.split('/'))
    # 68 ==> 1968, 07 ==> 2007
    y = y+1900 if y > 20 else y+2000
    return pd.datetime(y, m, d)

schema= pd.Series({
    'Id': 'int64',
    'is_bad': 'int64',
    # 'emp_title': 'category',
    'emp_length': 'float64',
    'home_ownership': 'category',
    'annual_inc': 'float64',
    'verification_status': 'category',
    # 'pymnt_plan': 'category',
    # 'Notes': 'category',
    'purpose_cat': 'category',
    # 'purpose': 'category',
    'zip_code': 'int64',
    'addr_state': 'category',
    'debt_to_income': 'float64',
    'delinq_2yrs': 'float64',
    'earliest_cr_line': 'datetime64[ns]',
    'inq_last_6mths': 'float64',
    'mths_since_last_delinq': 'float64',
    'mths_since_last_record': 'float64',
    'open_acc': 'float64',
    'pub_rec': 'float64',
    'revol_bal': 'float64',
    'revol_util': 'float64',
    'total_acc': 'float64',
    # 'initial_list_status': 'category',
    # 'collections_12_mths_ex_med': 'Int64',
    'mths_since_last_major_derog': 'int64',
    'policy_code': 'category'
})

usecols=schema.index
# zip_code and earliest_cr_line will be parsed through converters.
dtype=schema[~schema.index.isin(['zip_code', 'earliest_cr_line'])].to_dict()

lcd = pd.read_csv('./Lending Club Data - DR_Demo_Lending_Club.tsv', 
    sep='\t', 
    index_col='Id',
    na_values={'emp_length': 'na'},
    keep_default_na=True,
    usecols=usecols,
    dtype=dtype,
    converters={'zip_code':z3, 'earliest_cr_line':dateparse}
)

lcd['emp_length'].fillna(1, inplace=True)
lcd['emp_length'].clip(upper=10, inplace=True)
lcd['emp_length']=lcd['emp_length'].astype('int64')
# lcd['emp_length'].value_counts(dropna=False)


# lcd['zip_code'].dtype
# dtype('int64')

lcd['home_ownership'].replace('NONE', 'OTHER', inplace=True)
# lcd['home_ownership'].value_counts(dropna=False)

# lcd['annual_inc'].median()
lcd['annual_inc'].fillna(58000, inplace=True)
lcd['annual_inc'].clip(upper=250000, inplace=True)
# lcd['annual_inc'].value_counts(dropna=False)

sb_flag=lcd['purpose_cat'].str[-14:]=='small business'
lcd.loc[sb_flag, 'purpose_cat'] = 'small business'
# np.where(sb_flag, 'small business', lcd['purpose_cat'])
# lcd['purpose_cat'].value_counts(dropna=False)


lcd['delinq_2yrs'].fillna(0, inplace=True)
lcd['delinq_2yrs'].clip(upper=3, inplace=True)
lcd['delinq_2yrs']=lcd['delinq_2yrs'].astype('int64')
# lcd['delinq_2yrs'].value_counts(dropna=False)

lcd['inq_last_6mths'].fillna(0, inplace=True)
lcd['inq_last_6mths'].clip(upper=4, inplace=True)
lcd['inq_last_6mths']=lcd['inq_last_6mths'].astype('int64')
# lcd['inq_last_6mths'].value_counts(dropna=False)
# lcd.groupby('inq_last_6mths', observed=True).agg({'is_bad': ['sum', 'count']})

lcd['mths_since_last_delinq'].fillna(120.0, inplace=True)
# lcd.groupby('mths_since_last_delinq', observed=True).agg({'is_bad': ['sum', 'count']})

lcd['mths_since_last_record'].fillna(0.0, inplace=True)
lcd['inq_last_6mths']=lcd['inq_last_6mths'].astype('int64')
# lcd.groupby('mths_since_last_record', observed=True).agg({'is_bad': ['sum', 'count']})

lcd['open_acc'].fillna(7, inplace=True)
lcd['open_acc']=lcd['open_acc'].astype('int64')
# lcd['open_acc'].value_counts(dropna=False)
# lcd.groupby('open_acc', observed=True).agg({'is_bad': ['sum', 'count']})

lcd['pub_rec'].fillna(0, inplace=True)
lcd['pub_rec'].clip(upper=1, inplace=True)
lcd['pub_rec']=lcd['pub_rec'].astype('int64')
# lcd['pub_rec'].value_counts(dropna=False)
# lcd.groupby('pub_rec', observed=True).agg({'is_bad': ['sum', 'count']})

lcd['revol_bal'].clip(upper=100000, inplace=True)
# lcd.groupby('revol_bal', observed=True).agg({'is_bad': ['sum', 'count']})

lcd['revol_util'].fillna(0, inplace=True)
# lcd.groupby('revol_util', observed=True).agg({'is_bad': ['sum', 'count']})

lcd['total_acc'].fillna(1, inplace=True)
lcd['total_acc']=lcd['total_acc'].astype('int64')
# lcd['total_acc'].value_counts(dropna=False)
# lcd.groupby('total_acc', observed=True).agg({'is_bad': ['sum', 'count']})

# lcd.groupby('collections_12_mths_ex_med').agg({'is_bad': ['sum', 'count']})


Appending to ./lending_club_hp_tuning/train.py


In [7]:
%%writefile -a ./lending_club_hp_tuning/train.py
target = 'is_bad'
numerical_features = lcd.select_dtypes(include=['number']).columns
numerical_features = numerical_features.drop([target])

categorical_features =lcd.select_dtypes(include=['category']).columns
predictors=numerical_features.union(categorical_features, sort=False)

# exec(open("/mnt/c/Users/bjaco/Documents/projects_2020/mavenwave/python/lcd_1/lcd_read_input_1.py").read())
# ecl=pd.concat([lcd['earliest_cr_line'], lcd[target]], axis=1, copy=True)
# ecl.set_index('earliest_cr_line', inplace=True)
# ecl_q = ecl.groupby(by=pd.Grouper(freq='Q')).agg({target: ['sum', 'count']})
# pd.set_option('display.max_rows', 100)
# print(ecl_q[-100:])

#                  is_bad
#                     sum count
# earliest_cr_line
# 2006-09-30           10    87
# 2006-12-31           16    82
# 2007-03-31           12    64
# 2007-06-30           11    53
# 2007-09-30            7    39
# 2007-12-31            3    24
# 2008-03-31            1    21
# 2008-06-30            2     9
# 2008-09-30            0     3
# 2008-12-31            0     1

lcd.reset_index('Id', inplace=True)
lcd.set_index('earliest_cr_line', inplace=True)
start_train='2002-7-1'
start_test='2006-7-1'
end_test='2007-7-1'

lcd_train = lcd.loc[start_train:start_test].copy()
lcd_test = lcd.loc[start_test:end_test].copy()


lcd_train.reset_index('earliest_cr_line', drop=True, inplace=True)
lcd_test.reset_index('earliest_cr_line', drop=True, inplace=True)

# lcd.set_index(['Id'], append=True, inplace=True)
lcd_train.set_index(['Id'], inplace=True)
lcd_test.set_index(['Id'], inplace=True)

# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

X_train=lcd_train[predictors].copy()
y_train=lcd_train[target].copy()
# X_train.head()

X_test=lcd_test[predictors].copy()
y_test=lcd_test[target].copy()

Appending to ./lending_club_hp_tuning/train.py


In [8]:
%%writefile -a ./lending_club_hp_tuning/train.py
# Manually one-hot-encode categorical variables.
# https://towardsdatascience.com/categorical-encoding-using-label-encoding-and-one-hot-encoder-911ef77fb5bd
for cat_var in list(categorical_features.values):
    cat_col=pd.DataFrame(X_train[cat_var], columns=[cat_var])
    dum_df = pd.get_dummies(cat_col, columns=[cat_var], prefix=cat_var)
    X_train.drop(cat_var, axis=1, inplace=True)
    X_train=X_train.join(dum_df)
# X_train.columns

for cat_var in list(categorical_features.values):
    cat_col=pd.DataFrame(X_test[cat_var], columns=[cat_var])
    dum_df = pd.get_dummies(cat_col, columns=[cat_var], prefix=cat_var)
    X_test.drop(cat_var, axis=1, inplace=True)
    X_test=X_test.join(dum_df)
# X_test.columns

Appending to ./lending_club_hp_tuning/train.py


### Use the Hyperparameters
Use the Hyperparameter values passed in those arguments to set the corresponding hyperparameters in your application's XGBoost code.

In [9]:
%%writefile -a ./lending_club_hp_tuning/train.py

# Create the classifier, here we will use an XGboost classifier to demonstrate the use of HP Tuning.
# Here is where we set the variables used during HP Tuning from
# the parameters passed into the python script
classifier = xgb.XGBClassifier(max_depth=args.max_depth,
                             num_boost_round=args.num_boost_round,
                             booster=args.booster,
                             eval_metric='auc'
                            )

# Transform the features and fit them to the classifier
# classifier.fit(train_df[FEATURES], train_df[TARGET])
classifier.fit(X_train, y_train)



Appending to ./lending_club_hp_tuning/train.py


### Report the mean accuracy as hyperparameter tuning objective metric.

In [10]:
%%writefile -a ./lending_club_hp_tuning/train.py

# Calculate the mean accuracy on the given test data and labels.
score = classifier.score(X_test, y_test)

# The default name of the metric is training/hptuning/metric. 
# We recommend that you assign a custom name. The only functional difference is that 
# if you use a custom name, you must set the hyperparameterMetricTag value in the 
# HyperparameterSpec object in your job request to match your chosen name.
# https://cloud.google.com/ml-engine/reference/rest/v1/projects.jobs#HyperparameterSpec
hpt = hypertune.HyperTune()
hpt.report_hyperparameter_tuning_metric(
    hyperparameter_metric_tag='my_metric_tag',
    metric_value=score,
    global_step=100)



Appending to ./lending_club_hp_tuning/train.py


### Export and save the model to GCS

In [11]:
%%writefile -a ./lending_club_hp_tuning/train.py

# Export the model to a file
model_filename = 'model.pkl'
with open(model_filename, "wb") as f:
    pickle.dump(classifier, f)

# Example: job_dir = 'gs://BUCKET_ID/xgboost_job_dir/1'
job_dir =  args.job_dir.replace('gs://', '')  # Remove the 'gs://'
# Get the Bucket Id
bucket_id = job_dir.split('/')[0]
# Get the path
bucket_path = job_dir[len('{}/'.format(bucket_id)):]  # Example: 'xgboost_job_dir/1'
# (bucket_id, _, bucket_path) = job_dir.partition('/')

# Upload the model to GCS
bucket = storage.Client().bucket(bucket_id)
blob = bucket.blob('{}/{}'.format(
    bucket_path,
    model_filename))

blob.upload_from_filename(model_filename)



Appending to ./lending_club_hp_tuning/train.py


# Part 2: Create Trainer Package with Hyperparameter Tuning
Next we need to build the Trainer Package, which holds all your code and dependencies need to train your model on AI Platform. 

First, we create an empty `__init__.py` file.

In [12]:
%%writefile ./lending_club_hp_tuning/__init__.py

#!/usr/bin/env python

# Note that __init__.py can be an empty file.


Overwriting ./lending_club_hp_tuning/__init__.py


Next, we need to set the hp tuning values used to train our model. Check [HyperparameterSpec](https://cloud.google.com/ml-engine/reference/rest/v1/projects.jobs#HyperparameterSpec) for more info. 

In this config file several key things are set:
* `maxTrials` - How many training trials should be attempted to optimize the specified hyperparameters.
* `maxParallelTrials: 5` - The number of training trials to run concurrently. 
* `params` - The set of parameters to tune. These are the different parameters to pass into your model and the specified ranges you wish to try.
 * `parameterName` - The parameter name must be unique amongst all ParameterConfigs
 * `type` - The type of the parameter. [INTEGER, DOUBLE, ...]
 * `minValue` & `maxValue` - The range of values that this parameter could be. 
 * `scaleType` - How the parameter should be scaled to the hypercube. Leave unset for categorical parameters. Some kind of scaling is strongly recommended for real or integral parameters (e.g., UNIT_LINEAR_SCALE).

In [13]:
%%writefile ./hptuning_config.yaml

#!/usr/bin/env python

# Copyright 2018 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# hyperparam.yaml
trainingInput:
  hyperparameters:
    goal: MAXIMIZE
    maxTrials: 5
    maxParallelTrials: 5
    hyperparameterMetricTag: my_metric_tag
    enableTrialEarlyStopping: TRUE 
    params:
    - parameterName: max_depth
      type: INTEGER
      minValue: 3
      maxValue: 8
    - parameterName: num_boost_round
      type: INTEGER
      minValue: 50
      maxValue: 200
    - parameterName: booster
      type: CATEGORICAL
      categoricalValues: [
          "gbtree",
          "gblinear",
          "dart"
      ]



Overwriting ./hptuning_config.yaml


Lastly, we need to install the dependencies used in our model. Check [adding_standard_pypi_dependencies](https://cloud.google.com/ml-engine/docs/tensorflow/packaging-trainer#adding_standard_pypi_dependencies) for more info.

To do this, AI Platform uses a setup.py file to install your dependencies.

In [14]:
%%writefile ./setup.py

#!/usr/bin/env python

# Copyright 2018 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

from setuptools import find_packages
from setuptools import setup

REQUIRED_PACKAGES = ['cloudml-hypertune']

setup(
    name='lending_club_hp_tuning',
    version='0.1',
    install_requires=REQUIRED_PACKAGES,
    packages=find_packages(),
    include_package_data=True,
    description='Lending Club HP tuning training application'
)


Overwriting ./setup.py


# Part 3: Submit Training Job
Next we need to submit the job for training on AI Platform. We'll use gcloud to submit the job which has the following flags:

* `job-name` - A name to use for the job (mixed-case letters, numbers, and underscores only, starting with a letter). In this case: `auto_mpg_hp_tuning_$(date +"%Y%m%d_%H%M%S")`
* `job-dir` - The path to a Google Cloud Storage location to use for job output.
* `package-path` - A packaged training application that is staged in a Google Cloud Storage location. If you are using the gcloud command-line tool, this step is largely automated.
* `module-name` - The name of the main module in your trainer package. The main module is the Python file you call to start the application. If you use the gcloud command to submit your job, specify the main module name in the --module-name argument. Refer to Python Packages to figure out the module name.
* `region` - The Google Cloud Compute region where you want your job to run. You should run your training job in the same region as the Cloud Storage bucket that stores your training data. Select a region from [here](https://cloud.google.com/ml-engine/docs/regions) or use the default '`us-central1`'.
* `runtime-version` - The version of AI Platform to use for the job. If you don't specify a runtime version, the training service uses the default AI Platform runtime version 1.0. See the list of runtime versions for more information.
* `python-version` - The Python version to use for the job. Python 3.5 is available with runtime version 1.4 or greater. If you don't specify a Python version, the training service uses Python 2.7.
* `scale-tier` - A scale tier specifying the type of processing cluster to run your job on. This can be the CUSTOM scale tier, in which case you also explicitly specify the number and type of machines to use.
* `config` - Path to the job configuration file. This file should be a YAML document (JSON also accepted) containing a Job resource as defined in the API

Note: Check to make sure gcloud is set to the current PROJECT_ID

In [15]:
! gcloud config set project $PROJECT_ID

Updated property [core/project].


### Submit the training job.

In [16]:
now = !date +"%Y%m%d_%H%M%S"
%env JOB_NAME=black_friday_job_$now.s

!gcloud ai-platform jobs submit training $JOB_NAME \
  --job-dir gs://${BUCKET_ID}/data \
  --package-path $TRAINER_PACKAGE_PATH \
  --module-name $MAIN_TRAINER_MODULE \
  --region $REGION \
  --runtime-version=$RUNTIME_VERSION \
  --python-version=$PYTHON_VERSION \
  --scale-tier basic \
  --config $HPTUNING_CONFIG \
  -- \
  --project-id $PROJECT_ID \
  --bucket-name ${BUCKET_ID}    
    
# Stream logs so that training is done before subsequent cells are run.
# Remove  '> /dev/null' to see step-by-step output of the model build steps.
!gcloud ai-platform jobs stream-logs $JOB_NAME > /dev/null

# Model should exit with status "SUCCEEDED"
!gcloud ai-platform jobs describe $JOB_NAME --format="value(state)"

env: JOB_NAME=black_friday_job_20200614_204341
Job [black_friday_job_20200614_204341] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe black_friday_job_20200614_204341

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs black_friday_job_20200614_204341
jobId: black_friday_job_20200614_204341
state: QUEUED
SUCCEEDED


# [Optional] StackDriver Logging
You can view the logs for your training job:
1. Go to https://console.cloud.google.com/
1. Select "Logging" in left-hand pane
1. In left-hand pane, go to "AI Platform" and select Jobs
1. In filter by prefix, use the value of $JOB_NAME to view the logs

On the logging page of your model, you can view the different results for each HP tuning job. 

Example:
```
{
  "trialId": "15",
  "hyperparameters": {
    "booster": "dart",
    "max_depth": "7",
    "n_estimators": "102"
  },
  "finalMetric": {
    "trainingStep": "1000",
    "objectiveValue": 0.9259230441279733
  }
}
```

# [Optional] Verify Model File in GCS
View the contents of the destination model folder to verify that all 30 model files have indeed been uploaded to GCS.

Note: The model can take a few minutes to train and show up in GCS.

An example of the log output at the end of hyperparameter training showing the final AUC score and parameters chosen to achieve it:
